<a href="https://colab.research.google.com/github/DhruvaBansal00/CV-Spring2020-Project/blob/main/BiT_Cifar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/google-research/big_transfer.git

Cloning into 'big_transfer'...
remote: Enumerating objects: 90, done.
remote: Total 90 (delta 0), reused 0 (delta 0), pack-reused 90
Unpacking objects: 100% (90/90), done.


In [2]:
%cd big_transfer

/content/big_transfer


In [3]:
!pip install flax numpy tensorflow-cpu tensorflow-datasets tensorflow-probability

     |████████████████████████████████| 153kB 3.0MB/s 
     |████████████████████████████████| 137.3MB 41kB/s 


In [4]:
import os
if 'google.colab' in str(get_ipython()) and 'COLAB_TPU_ADDR' in os.environ:
  import requests
  if 'TPU_DRIVER_MODE' not in globals():
    url = 'http://' + os.environ['COLAB_TPU_ADDR'].split(':')[0] + ':8475/requestversion/tpu_driver0.1-dev20191206'
    resp = requests.post(url)
    TPU_DRIVER_MODE = 1
  from jax.config import config
  config.FLAGS.jax_xla_backend = "tpu_driver"
  config.FLAGS.jax_backend_target = "grpc://" + os.environ['COLAB_TPU_ADDR']
  print('Registered TPU:', config.FLAGS.jax_backend_target)
else:
  print('No TPU detected. Can be changed under "Runtime/Change runtime type".')

Registered TPU: grpc://10.13.37.42:8470


In [5]:
!curl -OL https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz
!wget https://storage.googleapis.com/bit_models/BiT-M-R152x2.npz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  162M  100  162M    0     0  85.8M      0  0:00:01  0:00:01 --:--:-- 85.8M
--2020-10-31 17:40:35--  https://storage.googleapis.com/bit_models/BiT-M-R152x2.npz
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.125.128, 209.85.200.128, 74.125.124.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.125.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1287051752 (1.2G) [application/octet-stream]
Saving to: ‘BiT-M-R152x2.npz’

BiT-M-R152x2.npz    100%[===================>]   1.20G  43.3MB/s    in 21s     

2020-10-31 17:40:56 (58.7 MB/s) - ‘BiT-M-R152x2.npz’ saved [1287051752/1287051752]



In [6]:
!curl -OL https://www.cs.toronto.edu/~kriz/cifar-100-python.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  161M  100  161M    0     0  67.5M      0  0:00:02  0:00:02 --:--:-- 67.5M


In [7]:
import logging
import logging.config
def setup_logger(args):
  """Creates and returns a fancy logger."""
  # return logging.basicConfig(level=logging.INFO, format="[%(asctime)s] %(message)s")
  # Why is setting up proper logging so !@?#! ugly?
  os.makedirs(os.path.join(args['logdir'], args['name']), exist_ok=True)
  logging.config.dictConfig({
      "version": 1,
      "disable_existing_loggers": False,
      "formatters": {
          "standard": {
              "format": "%(asctime)s [%(levelname)s] %(name)s: %(message)s"
          },
      },
      "handlers": {
          "stderr": {
              "level": "INFO",
              "formatter": "standard",
              "class": "logging.StreamHandler",
              "stream": "ext://sys.stderr",
          },
          "logfile": {
              "level": "DEBUG",
              "formatter": "standard",
              "class": "logging.FileHandler",
              "filename": os.path.join(args['logdir'], args['name'], "train.log"),
              "mode": "a",
          }
      },
      "loggers": {
          "": {
              "handlers": ["stderr", "logfile"],
              "level": "DEBUG",
              "propagate": True
          },
      }
  })
  logger = logging.getLogger(__name__)
  logger.flush = lambda: [h.flush() for h in logger.handlers]
  logger.info(args)
  return logger

In [8]:
import tensorflow as tf
import tensorflow_probability as tfp
import tensorflow_datasets as tfds

import numpy as np

import resource
low, high = resource.getrlimit(resource.RLIMIT_NOFILE)
resource.setrlimit(resource.RLIMIT_NOFILE, (high, high))

MAX_IN_MEMORY = 200_000


DATASET_SPLITS = {
  'cifar10': {'train': 'train', 'test': 'test'},
  'cifar100': {'train': 'train', 'test': 'test'},
  'imagenet2012': {'train': 'train[:99%]', 'test': 'validation'},
}


def get_dataset_info(dataset, split, examples_per_class):
  data_builder = tfds.builder(dataset)
  original_num_examples = data_builder.info.splits[split].num_examples
  num_classes = data_builder.info.features['label'].num_classes
  if examples_per_class is not None:
    num_examples = examples_per_class * num_classes
  else:
    num_examples = original_num_examples
  return {'original_num_examples': original_num_examples,
          'num_examples': num_examples,
          'num_classes': num_classes}


def sample_subset(data, num_examples, num_classes,
                  examples_per_class, examples_per_class_seed):
  data = data.batch(min(num_examples, MAX_IN_MEMORY))

  data = data.as_numpy_iterator().next()

  np.random.seed(examples_per_class_seed)
  indices = [idx
             for c in range(num_classes)
             for idx in np.random.choice(np.where(data['label'] == c)[0],
                                         examples_per_class,
                                         replace=False)]

  data = {'image': data['image'][indices],
          'label': data['label'][indices]}

  data = tf.data.Dataset.zip(
    (tf.data.Dataset.from_tensor_slices(data['image']),
     tf.data.Dataset.from_tensor_slices(data['label'])))
  return data.map(lambda x, y: {'image': x, 'label': y},
                  tf.data.experimental.AUTOTUNE)


def get_data(dataset, mode,
             repeats, batch_size,
             resize_size, crop_size,
             mixup_alpha,
             examples_per_class, examples_per_class_seed,
             num_devices,
             tfds_manual_dir):

  split = DATASET_SPLITS[dataset][mode]
  dataset_info = get_dataset_info(dataset, split, examples_per_class)

  data_builder = tfds.builder(dataset)
  data_builder.download_and_prepare(
   download_config=tfds.download.DownloadConfig(manual_dir=tfds_manual_dir))
  data = data_builder.as_dataset(
    split=split,
    decoders={'image': tfds.decode.SkipDecoding()})
  decoder = data_builder.info.features['image'].decode_example

  if (mode == 'train') and (examples_per_class is not None):
    data = sample_subset(data,
                         dataset_info['original_num_examples'],
                         dataset_info['num_classes'],
                         examples_per_class, examples_per_class_seed)

  def _pp(data):
    im = decoder(data['image'])
    if mode == 'train':
      im = tf.image.resize(im, [resize_size, resize_size])
      im = tf.image.random_crop(im, [crop_size, crop_size, 3])
     # im = tf.image.flip_left_right(im)
    else:
      # usage of crop_size here is intentional
      im = tf.image.resize(im, [crop_size, crop_size])
    im = (im - 127.5) / 127.5
    label = tf.one_hot(data['label'], dataset_info['num_classes'])
    return {'image': im, 'label': label}

  data = data.cache()
  data = data.repeat(repeats)
  if mode == 'train':
    data = data.shuffle(min(dataset_info['num_examples'], MAX_IN_MEMORY))
  data = data.map(_pp, tf.data.experimental.AUTOTUNE)
  data = data.batch(batch_size, drop_remainder=True)

  def _mixup(data):
    beta_dist = tfp.distributions.Beta(mixup_alpha, mixup_alpha)
    beta = tf.cast(beta_dist.sample([]), tf.float32)
    data['image'] = (beta * data['image'] +
                     (1 - beta) * tf.reverse(data['image'], axis=[0]))
    data['label'] = (beta * data['label'] +
                     (1 - beta) * tf.reverse(data['label'], axis=[0]))
    return data

  if mixup_alpha is not None and mixup_alpha > 0.0 and mode == 'train':
    data = data.map(_mixup, tf.data.experimental.AUTOTUNE)

  # Shard data such that it can be distributed accross devices
  def _shard(data):
    data['image'] = tf.reshape(data['image'],
                               [num_devices, -1, crop_size, crop_size, 3])
    data['label'] = tf.reshape(data['label'],
                               [num_devices, -1, dataset_info['num_classes']])
    return data
  if num_devices is not None:
    data = data.map(_shard, tf.data.experimental.AUTOTUNE)

  return data.prefetch(1)

In [ ]:
# import os

# from functools import partial

# import numpy as np

# import jax
# import jax.numpy as jnp

# import flax.optim as optim
# import flax.jax_utils as flax_utils

# import bit_jax.models as models
# import bit_jax.tf2jax as tf2jax

# import bit_common
# import bit_hyperrule

# tfds_manual_dir = None
# batch_eval = 32
# modelName = "BiT-M-R152x2" 
# dataset = "cifar10"
# examples_per_class = None
# batch = 512 
# eval_every = 10
# args = {
#     "logdir" : "tmp/bit_logs",
#     "name" : "cifar10_`date +%F_%H%M%S`"
# }
# base_lr  = 0.003
# logger = setup_logger(args)

# logger.info(f'Available devices: {jax.devices()}')

# model = models.KNOWN_MODELS[modelName]

# # Load weigths of a BiT model
# bit_model_file = f'{modelName}.npz'
# print(bit_model_file)
# if not os.path.exists(bit_model_file):
#   raise FileNotFoundError() 
# with open(bit_model_file, 'rb') as f:
#   params_tf = np.load(f)
#   params_tf = dict(zip(params_tf.keys(), params_tf.values()))

# resize_size, crop_size = bit_hyperrule.get_resolution_from_dataset(
#   dataset)

# # Setup input pipeline
# dataset_info = get_dataset_info(
#   dataset, 'train', examples_per_class)

# data_train = get_data(
#   dataset=dataset,
#   mode='train',
#   repeats=None, batch_size=batch,
#   resize_size=resize_size, crop_size=crop_size,
#   examples_per_class=examples_per_class,
#   examples_per_class_seed=None,
#   mixup_alpha=None,
#   num_devices=jax.local_device_count(),
#   tfds_manual_dir=tfds_manual_dir)
# logger.info(data_train)
# data_test = get_data(
#   dataset=dataset,
#   mode='test',
#   repeats=1, batch_size=batch_eval,
#   resize_size=resize_size, crop_size=crop_size,
#   examples_per_class=None, examples_per_class_seed=0,
#   mixup_alpha=None,
#   num_devices=jax.local_device_count(),
#   tfds_manual_dir=tfds_manual_dir)
# logger.info(data_test)

# # Build ResNet architecture
# ResNet = model.partial(num_classes=dataset_info['num_classes'])
# _, params = ResNet.init_by_shape(
#   jax.random.PRNGKey(0),
#   [([1, crop_size, crop_size, 3], jnp.float32)])
# resnet_fn = ResNet.call

# # pmap replicates the models over all GPUs
# resnet_fn_repl = jax.pmap(ResNet.call)

# def cross_entropy_loss(*, logits, labels):
#   logp = jax.nn.log_softmax(logits)
#   return -jnp.mean(jnp.sum(logp * labels, axis=1))

# def loss_fn(params, images, labels):
#   logits = resnet_fn(params, images)
#   return cross_entropy_loss(logits=logits, labels=labels)

# # Update step, replicated over all GPUs
# @partial(jax.pmap, axis_name='batch')
# def update_fn(opt, lr, batch):
#   l, g = jax.value_and_grad(loss_fn)(opt.target,
#                                       batch['image'],
#                                       batch['label'])
#   g = jax.tree_map(lambda x: jax.lax.pmean(x, axis_name='batch'), g)
#   opt = opt.apply_gradient(g, learning_rate=lr)
#   return opt

# # In-place update of randomly initialized weights by BiT weigths
# tf2jax.transform_params(params, params_tf,
#                         num_classes=dataset_info['num_classes'])

# # Create optimizer and replicate it over all GPUs
# opt = optim.Momentum(beta=0.9).create(params)
# opt_repl = flax_utils.replicate(opt)

# # Delete referenes to the objects that are not needed anymore
# del opt
# del params

# total_steps = bit_hyperrule.get_schedule(dataset_info['num_examples'])[-1]

# # Run training loop
# for step, batch in zip(range(1, total_steps + 1),
#                         data_train.as_numpy_iterator()):
#   lr = bit_hyperrule.get_lr(step - 1,
#                             dataset_info['num_examples'],
#                             base_lr)
#   opt_repl = update_fn(opt_repl, flax_utils.replicate(lr), batch)

#   # Run eval step
#   if ((eval_every and step % eval_every == 0)
#         or (step == total_steps)):

#     accuracy_test = np.mean([
#       c
#       for batch in data_test.as_numpy_iterator()
#       for c in (
#         np.argmax(resnet_fn_repl(opt_repl.target, batch['image']), axis=2) ==
#         np.argmax(batch['label'], axis=2)).ravel()])

#     logger.info(
#             f'Step: {step}, '
#             f'learning rate: {lr:.07f}, '
#             f'Test accuracy: {accuracy_test:0.3f}')


Train Set size vs Accuracy

In [17]:
import os
from functools import partial
import numpy as np
import jax
import jax.numpy as jnp
import flax.optim as optim
import flax.jax_utils as flax_utils
import bit_jax.models as models
import bit_jax.tf2jax as tf2jax
import bit_common
import bit_hyperrule

def cross_entropy_loss(*, logits, labels):
  logp = jax.nn.log_softmax(logits)
  return -jnp.mean(jnp.sum(logp * labels, axis=1))

def loss_fn(params, images, labels):
  logits = resnet_fn(params, images)
  return cross_entropy_loss(logits=logits, labels=labels)

# Update step, replicated over all GPUs
@partial(jax.pmap, axis_name='batch')
def update_fn(opt, lr, batch):
  l, g = jax.value_and_grad(loss_fn)(opt.target,
                                      batch['image'],
                                      batch['label'])
  g = jax.tree_map(lambda x: jax.lax.pmean(x, axis_name='batch'), g)
  opt = opt.apply_gradient(g, learning_rate=lr)
  return opt

examples_per_class_range = [1, 5, 50, 500]
# examples_per_class_range.extend([5000, None])
# examples_per_class_range = examples_per_class_range[::-1]
print(examples_per_class_range)
total_steps = 100
# bit_hyperrule.get_schedule(dataset_info['num_examples'])[-1]
# print(total_steps)

accuracies = []

for curr_examples in examples_per_class_range:
    # Setup input pipeline
  tfds_manual_dir = None
  batch_eval = 32
  modelName = "BiT-M-R152x2" 
  dataset = "cifar100"
  examples_per_class = None
  batch = 128 
  eval_every = 10
  args = {
      "logdir" : "tmp/bit_logs",
      "name" : "cifar100_`date +%F_%H%M%S`"
  }
  base_lr  = 0.003
  logger = setup_logger(args)

  logger.info(f'Available devices: {jax.devices()}')

  model = models.KNOWN_MODELS[modelName]

  # Load weigths of a BiT model
  bit_model_file = f'{modelName}.npz'
  print(bit_model_file)
  if not os.path.exists(bit_model_file):
    raise FileNotFoundError() 
  with open(bit_model_file, 'rb') as f:
    params_tf = np.load(f)
    params_tf = dict(zip(params_tf.keys(), params_tf.values()))

  resize_size, crop_size = bit_hyperrule.get_resolution_from_dataset(
    dataset)
  dataset_info = get_dataset_info(
    dataset, 'train', None)
  print(dataset_info)

  data_train = get_data(
    dataset=dataset,
    mode='train',
    repeats=None, batch_size=batch,
    resize_size=resize_size, crop_size=crop_size,
    examples_per_class=curr_examples,
    examples_per_class_seed=None,
    mixup_alpha=None,
    num_devices=jax.local_device_count(),
    tfds_manual_dir=tfds_manual_dir)
  logger.info(data_train)
  data_test = get_data(
    dataset=dataset,
    mode='test',
    repeats=1, batch_size=batch_eval,
    resize_size=resize_size, crop_size=crop_size,
    examples_per_class=None, examples_per_class_seed=0,
    mixup_alpha=None,
    num_devices=jax.local_device_count(),
    tfds_manual_dir=tfds_manual_dir)
  logger.info(data_test)

  # Build ResNet architecture
  ResNet = model.partial(num_classes=dataset_info['num_classes'])
  _, params = ResNet.init_by_shape(
    jax.random.PRNGKey(0),
    [([1, crop_size, crop_size, 3], jnp.float32)])
  resnet_fn = ResNet.call

  # pmap replicates the models over all GPUs
  resnet_fn_repl = jax.pmap(ResNet.call)

  # In-place update of randomly initialized weights by BiT weigths
  tf2jax.transform_params(params, params_tf,
                          num_classes=dataset_info['num_classes'])

  # Create optimizer and replicate it over all GPUs
  opt = optim.Momentum(beta=0.9).create(params)
  opt_repl = flax_utils.replicate(opt)

  # Delete referenes to the objects that are not needed anymore
  del opt
  del params


  accuracy_test = 0
  # Run training loop
  for step, batch in zip(range(1, total_steps + 1),
                          data_train.as_numpy_iterator()):
    lr = bit_hyperrule.get_lr(step - 1,
                              dataset_info['num_examples'],
                              base_lr)
    opt_repl = update_fn(opt_repl, flax_utils.replicate(lr), batch)

    # Run eval step
    if ((eval_every and step % eval_every == 0)
          or (step == total_steps)):

      accuracy_test = np.mean([
        c
        for batch in data_test.as_numpy_iterator()
        for c in (
          np.argmax(resnet_fn_repl(opt_repl.target, batch['image']), axis=2) ==
          np.argmax(batch['label'], axis=2)).ravel()])

      logger.info(
              f'Step: {step}, '
              f'learning rate: {lr:.07f}, '
              f'Test accuracy: {accuracy_test:0.3f}')


  accuracies.append(accuracy_test)
  print(accuracies)



2020-10-31 21:23:48,714 [INFO] __main__: {'logdir': 'tmp/bit_logs', 'name': 'cifar100_`date +%F_%H%M%S`'}
2020-10-31 21:23:48,716 [INFO] __main__: Available devices: [TpuDevice(id=0, host_id=0, coords=(0,0,0), core_on_chip=0), TpuDevice(id=1, host_id=0, coords=(0,0,0), core_on_chip=0), TpuDevice(id=2, host_id=0, coords=(0,0,0), core_on_chip=0), TpuDevice(id=3, host_id=0, coords=(0,0,0), core_on_chip=0), TpuDevice(id=4, host_id=0, coords=(0,0,0), core_on_chip=0), TpuDevice(id=5, host_id=0, coords=(0,0,0), core_on_chip=0), TpuDevice(id=6, host_id=0, coords=(0,0,0), core_on_chip=0), TpuDevice(id=7, host_id=0, coords=(0,0,0), core_on_chip=0)]


[1, 5, 50, 500]
BiT-M-R152x2.npz


2020-10-31 21:23:52,748 [INFO] absl: Load pre-computed DatasetInfo (eg: splits, num examples,...) from GCS: cifar100/3.0.2
2020-10-31 21:23:52,940 [INFO] absl: Load dataset info from /tmp/tmpsbzhcuobtfds
2020-10-31 21:23:52,950 [INFO] absl: Field info.citation from disk and from code do not match. Keeping the one from code.
2020-10-31 21:23:52,957 [INFO] absl: Load pre-computed DatasetInfo (eg: splits, num examples,...) from GCS: cifar100/3.0.2
2020-10-31 21:23:53,143 [INFO] absl: Load dataset info from /tmp/tmp55ojw13atfds


{'original_num_examples': 50000, 'num_examples': 50000, 'num_classes': 100}


2020-10-31 21:23:53,161 [INFO] absl: Field info.citation from disk and from code do not match. Keeping the one from code.
2020-10-31 21:23:53,177 [INFO] absl: Load pre-computed DatasetInfo (eg: splits, num examples,...) from GCS: cifar100/3.0.2
2020-10-31 21:23:53,366 [INFO] absl: Load dataset info from /tmp/tmpntybs09xtfds
2020-10-31 21:23:53,379 [INFO] absl: Field info.citation from disk and from code do not match. Keeping the one from code.
2020-10-31 21:23:53,382 [INFO] absl: Generating dataset cifar100 (/root/tensorflow_datasets/cifar100/3.0.2)


...

[0.3879206730769231, 0.7085336538461539, 0.8168068910256411]
BiT-M-R152x2.npz


2020-10-31 21:56:30,031 [INFO] absl: Load dataset info from /root/tensorflow_datasets/cifar100/3.0.2
2020-10-31 21:56:30,043 [INFO] absl: Load dataset info from /root/tensorflow_datasets/cifar100/3.0.2
2020-10-31 21:56:30,053 [INFO] absl: Load dataset info from /root/tensorflow_datasets/cifar100/3.0.2
2020-10-31 21:56:30,067 [INFO] absl: Reusing dataset cifar100 (/root/tensorflow_datasets/cifar100/3.0.2)
2020-10-31 21:56:30,071 [INFO] absl: Constructing tf.data.Dataset for split train, from /root/tensorflow_datasets/cifar100/3.0.2


{'original_num_examples': 50000, 'num_examples': 50000, 'num_classes': 100}


2020-10-31 21:56:37,043 [INFO] __main__: <PrefetchDataset shapes: {image: (8, 16, 128, 128, 3), label: (8, 16, 100)}, types: {image: tf.float32, label: tf.float32}>
2020-10-31 21:56:37,060 [INFO] absl: Load dataset info from /root/tensorflow_datasets/cifar100/3.0.2
2020-10-31 21:56:37,076 [INFO] absl: Load dataset info from /root/tensorflow_datasets/cifar100/3.0.2
2020-10-31 21:56:37,088 [INFO] absl: Reusing dataset cifar100 (/root/tensorflow_datasets/cifar100/3.0.2)
2020-10-31 21:56:37,092 [INFO] absl: Constructing tf.data.Dataset for split test, from /root/tensorflow_datasets/cifar100/3.0.2
2020-10-31 21:56:37,500 [INFO] __main__: <PrefetchDataset shapes: {image: (8, 4, 128, 128, 3), label: (8, 4, 100)}, types: {image: tf.float32, label: tf.float32}>
2020-10-31 21:59:21,144 [INFO] __main__: Step: 10, learning rate: 0.0000540, Test accuracy: 0.056
2020-10-31 22:00:05,834 [INFO] __main__: Step: 20, learning rate: 0.0001140, Test accuracy: 0.111
2020-10-31 22:00:50,234 [INFO] __main__: 

[0.3879206730769231, 0.7085336538461539, 0.8168068910256411, 0.8357371794871795]
